# Dyna-Q & Model-Based Planning

## Model là gì? 

**Model** = Cách lưu trữ kiến thức về **transition dynamics** và **reward dynamics** của environment

### Tại sao cần model?
- Dùng cho **planning** - quá trình sử dụng model để cải thiện policy
- Cách planning: Dùng **Simulated Experience** + Value function updates
- **Logic**: Estimate value tốt hơn → decisions tốt hơn

**💡 Ý tưởng chính**: Thay vì chỉ học từ real experience, ta có thể "tưởng tượng" experience từ model!

## 2 Loại Model 

### 1. Sample Models
- **Chức năng**: Tạo ra **1 kết quả cụ thể** từ underlying probabilities
- **Ưu điểm**: 
  - ✅ **Computationally cheap** 
  - ✅ Dễ implement (chỉ cần random theo rules)
- **Ví dụ**: Throw a dice → ra số 4 (không cần biết P(1)=1/6, P(2)=1/6,...)

### 2. Distribution Models  
- **Chức năng**: Specify **probability của TẤT CẢ outcomes**
- **Ưu điểm**: 
  - ✅ **Chứa nhiều thông tin hơn**
  - ✅ Complete knowledge về environment
- **Nhược điểm**:
  - ❌ **Khó specify** (cần biết tất cả probabilities)
  - ❌ **Very large** (exponential với số states/actions)

**🎯 Trong practice**: Sample models được dùng nhiều hơn vì practical!

## Random Sample One-Step Q-Learning 

### Setup
**Giả định**:
- Có sample model của transition dynamics
- Có strategy để sample relevant state-action pairs

### Algorithm (3 steps)

1. **Random sampling**: 
   - Chọn random (state, action) pair từ tất cả possible pairs
   - Query sample model → get next state & reward

2. **Q-learning update**: 
   - Perform Q-learning update trên model transition này
   - `Q(s,a) ← Q(s,a) + α[r + γmax Q(s',a') - Q(s,a)]`

3. **Policy improvement**: 
   - Improve policy bằng cách **greedify** theo updated Q-values

**🔑 Key point**: Học từ **simulated experience** thay vì chỉ real experience!

## Dyna Architecture 

**Ý tưởng**: Kết hợp **learning** (từ real experience) + **planning** (từ simulated experience)

### Tabular Dyna-Q Algorithm

**Initialization**:
- `Q(s,a)` và `Model(s,a)` cho tất cả states & actions

**Main Loop (forever)**:

**🔴 DIRECT RL (học từ real experience)**:
- (a) `S ← current state` (non-terminal)
- (b) `A ← ε-greedy(S, Q)` 
- (c) Take action A; observe R, S'
- (d) **Q-update**: `Q(S,A) ← Q(S,A) + α[R + γmax Q(S',a) - Q(S,A)]`
- (e) **Model update**: `Model(S,A) ← R, S'` (assuming deterministic env)

**🔵 PLANNING (học từ simulated experience)**:
- (f) **Repeat n times**:
  - `S ← random previously observed state`
  - `A ← random action previously taken in S`  
  - `R, S' ← Model(S,A)` (simulate experience)
  - **Q-update**: `Q(S,A) ← Q(S,A) + α[R + γmax Q(S',a) - Q(S,A)]`

**💡 Điểm hay**: Mỗi real step → n planning steps! Tăng tốc learning đáng kể.

## Vấn đề: Model Không Chính Xác

### Khi nào model inaccurate?

**1. Incomplete Model**:
- **Nguyên nhân**: Agent chưa thử hầu hết actions ở hầu hết states
- **Hệ quả**: Transitions cho những (s,a) chưa thử = **missing**
- **Timeline**: Thường xảy ra đầu quá trình learning

**2. Changing Environment**:
- **Nguyên nhân**: Environment thay đổi theo thời gian
- **Hệ quả**: Same action ở same state → different outcome so với trước
- **Ví dụ**: Đường bị blocked, rules thay đổi, etc.

### Hệ quả của Inaccurate Model

⚠️ **Warning**: Planning với inaccurate model chỉ improve policy **đối với model**, **KHÔNG phải** environment thật!

→ Agent có thể học policy tốt cho "thế giới ảo" nhưng tệ cho thế giới thật

## Solutions cho Model Problems 

### 1. Incomplete Model
**Dyna-Q approach**: Chỉ sample những state-action pairs **đã được visit trước đó**
- ✅ **Safe**: Không plan với data không có
- ✅ **Practical**: Dùng experience có sẵn efficiently
- ❌ **Conservative**: Có thể miss những opportunities mới

### 2. Changing Environment: Dyna-Q+ 

**Ý tưởng**: Encourage agent **revisit** states periodically để detect changes

**Cách làm**: Thêm **bonus reward** vào planning:

```
New reward = r + κ√τ
```

**Trong đó**:
- `r`: actual reward
- `κ`: small constant (control influence)
- `τ`: time steps since transition was last tried

**Logic**: 
- Transition càng lâu không thử → τ càng lớn → bonus càng lớn
- Agent được khuyến khích explore những chỗ đã lâu không đến
- Nếu environment thay đổi → sẽ được detect sớm hơn

**🎯 Kết quả**: Dyna-Q+ = Dyna-Q + exploration bonus for planning

## Tổng Kết & Insights 

### Ưu điểm của Model-Based Methods
- ✅ **Sample efficiency**: Học từ cả real + simulated experience
- ✅ **Faster learning**: n planning steps per real step
- ✅ **Knowledge reuse**: Model có thể dùng cho nhiều tasks

### Challenges
- ❌ **Model accuracy**: Inaccurate model → poor performance
- ❌ **Computational cost**: Maintain & query model
- ❌ **Environment changes**: Cần detect & adapt

### Khi nào dùng Model-Based?
- **Environment stable**: Ít thay đổi, model reliable
- **Sample expensive**: Real experience khó/đắt để có
- **Planning horizon**: Cần think ahead nhiều steps

### Khi nào dùng Model-Free?
- **Environment dynamic**: Thay đổi thường xuyên
- **Simple tasks**: Không cần complex planning
- **Real-time**: Cần respond nhanh, không có time để plan

---

*Personal note: Dyna-Q là một breakthrough vì combine được benefits của cả model-based và model-free. Trong practice, nhiều state-of-the-art algorithms đều có elements của cả hai approaches. Key là biết khi nào trust model và khi nào không!*